In [1]:
from utt_encoding import UtteranceEncoding
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
import pandas as pd
from IPython.display import clear_output

In [2]:
def string_prop(dp_content):
    dp_content = dp_content.replace("!=", "does not equal").replace("=", "equals").\
    replace("<", "is less than").replace(">", "is more than").\
    replace("+", "plus").replace("10", "ten").replace("20", "twenty").replace("30", "thirty").\
    replace("40", "forty").replace("50","fifty").replace("red", "red block").replace("blue", "blue block").\
    replace("yellow", "yellow block").replace("purple", "purple block").replace("green", "green block")
    return dp_content

In [3]:
def unstring_prop(dp_content):
    dp_content = dp_content.replace("equals","=").replace("does not equal","!=").\
    replace("is less than","<").replace("is more than",">").\
    replace("plus","+").replace("ten","10").replace("twenty","20").replace("thirty","30").\
    replace("forty","40").replace("fifty","50").replace(" block","")
    return dp_content

In [6]:
props = list(pd.read_csv("NormalizedList.csv")["Propositions"])
for id, prop in enumerate(props):
    props[id] = string_prop(prop)

In [7]:
utterance_encoder = UtteranceEncoding(props)
clear_output(wait=False)

In [8]:
colors = ['red', 'blue', 'green', 'purple', 'yellow']

In [9]:
def calculate_iou(common_ground, actual_common_ground):
    # Split by ',' and strip whitespace from each element
    common_ground_elements = set([element.strip() for element in common_ground.split(',')])
    actual_common_ground_elements = set([element.strip() for element in actual_common_ground.split(',')])
    # Calculate intersection and union
    intersection = common_ground_elements & actual_common_ground_elements
    union = common_ground_elements | actual_common_ground_elements
    # Calculate IoU
    iou = len(intersection) / len(union) if union else 0
    return iou

In [32]:
l = []
top1 = []
top3 = []
topmax = []
iou = []
acc_1, acc_3, acc_top_max, acc_all = 0, 0, 0, 0
jacc_1, jacc_3, jacc_top_max, jacc_all =[], [], [], []
data_file = pd.read_csv("preprocessedTrainingData.csv")
data_file = data_file[data_file["Label"]==1]
spreadsheet = pd.DataFrame()

for row_id, utterance in enumerate(data_file["Transcript"]):
    selected_colors = []
    for color in colors:
        if color in utterance.lower():
            selected_colors.append(color)
    tokenized_text, seq_embedding, pooled_embedding, hidden_states = \
                            utterance_encoder.get_sentence_embedding(utterance,
                            utterance_encoder.bert_base_uncased_tokenizer,
                            utterance_encoder.bert_base_uncased_model)
    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0), dim=1).permute(1,0,2)
    sum_embeddings = utterance_encoder.get_token_embeddings(token_embeddings, 4)
    utterance_embedding = torch.mean(torch.stack(sum_embeddings[:-1]),axis=0)
    similarities = []
    for prop in utterance_encoder.prop_embs:
        # for color in selected_colors:
        #     if color in prop:
                similarities.append(cos_sim(utterance_embedding.reshape(1,-1),\
                    utterance_encoder.prop_embs[prop].reshape(1,-1))[0][0])
    # print(np.max(np.array(similarities)),utterance_encoder.props[np.argmax(np.array(similarities))])
    true_prop = data_file.iloc[row_id]["Common Ground"]
    # Top 1 acc
    dp_content = utterance_encoder.props[np.argmax(np.array(similarities))]
    dp_content = unstring_prop(dp_content)
    if dp_content == true_prop:
        acc_1 += 1
        top1.append(row_id)
    iou.append(calculate_iou(dp_content, true_prop))

    # Top 3 acc
    for prop_3 in np.array(similarities).argsort()[::-1][:3]:
        dp_content = utterance_encoder.props[prop_3]
        dp_content = unstring_prop(dp_content)
        if dp_content == true_prop:
            acc_3 += 1
            top3.append(row_id)
    # tmp = {"Utt_id":0, "Transcript":"", "Ground truth":"", "Correct":0, "Top 1 prop":"", "Top 1 cosim":0}
    # Top 5 acc
    tmp = {}
    tmp["Utt_id"] = row_id
    tmp["Transcript"] = utterance
    tmp["Ground truth"] = true_prop
    tmp["Correct"] = 0

    old = acc_top_max
    top_max = 10
    for i, prop_top_max in enumerate(np.array(similarities).argsort()[::-1][:top_max]):
        dp_content = utterance_encoder.props[prop_top_max]
        dp_content = unstring_prop(dp_content)
        if dp_content == true_prop:
            acc_top_max += 1
            topmax.append(row_id)
            tmp["Correct"] = i+1
        tmp["Top "+ str(i+1)+ " prop"] = dp_content
        tmp["Top "+ str(i+1)+ " cosim"] = np.sort(np.array(similarities))[::-1][:top_max][i]
        # add to tmp df the oracle transcript, the 5 DPs and their cosines
        tmp["Top "+ str(i+1)+ " IOU"] = calculate_iou(dp_content, true_prop)
    # add all metrics to corresponding utt 
    df_dictionary = pd.DataFrame([tmp])
    spreadsheet = pd.concat([spreadsheet, df_dictionary], ignore_index=True)
    if acc_top_max == old:
        l.append(row_id)

In [27]:
# spreadsheet.to_csv("DP_bert_10.csv", index=False)

In [33]:
print(top1)
print(top3)
print(topmax)

[1, 3, 6, 12, 30, 49, 50, 55, 57, 64, 67, 91, 110]
[0, 1, 3, 6, 12, 15, 21, 30, 32, 37, 49, 50, 55, 57, 64, 65, 67, 86, 91, 92, 94, 110]
[0, 1, 3, 6, 8, 9, 12, 15, 20, 21, 30, 32, 37, 42, 49, 50, 52, 55, 57, 59, 64, 65, 67, 74, 83, 86, 88, 89, 91, 92, 94, 102, 104, 107, 108, 110, 117, 125]


In [34]:
print(acc_1)
print(acc_3)
print(acc_top_max)

13
22
38


In [36]:
sum(iou)/len(iou)

0.12427821522309711